In [1]:
import numpy as np
import tensorflow as tf

In [2]:
idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
# x_data = [[0, 1, 0, 2, 3, 3]]  # hihell
y_data = [[1, 0, 2, 3, 3, 4]]  # ihello

In [3]:
num_classes = 5
input_dim = 5  # one-hot size, same as hidden_size to directly predict one-hot
sequence_length = 6  # |ihello| == 6
learning_rate = 0.1

In [4]:
x_one_hot = np.array([[[1, 0, 0, 0, 0],    # h 0
                       [0, 1, 0, 0, 0],    # i 1
                       [1, 0, 0, 0, 0],    # h 0
                       [0, 0, 1, 0, 0],    # e 2
                       [0, 0, 0, 1, 0],    # l 3
                       [0, 0, 0, 1, 0]]],  # l 3
                     dtype=np.float32)

y_one_hot = tf.keras.utils.to_categorical(y_data, num_classes=num_classes)
print(x_one_hot.shape)
print(y_one_hot.shape)

(1, 6, 5)
(1, 6, 5)


In [5]:
tf.model = tf.keras.Sequential()
# make cell and add it to RNN layer
# input_shape = (1,6,5) => number of sequence (batch), length of sequence, size of input dim
cell = tf.keras.layers.LSTMCell(units=num_classes, input_shape=(sequence_length, input_dim))
tf.model.add(tf.keras.layers.RNN(cell=cell, return_sequences=True))

# single LSTM layer can be used as well instead of creating LSTMCell
# tf.model.add(tf.keras.layers.LSTM(units=num_classes, input_shape=(sequence_length, input_dim), return_sequences=True))

# fully connected layer
tf.model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=num_classes, activation='softmax')))

tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                 metrics=['accuracy'])

In [6]:
# train
tf.model.fit(x_one_hot, y_one_hot, epochs=50)

Epoch 1/50
1/1 [==============================] - 0s 1ms/step - loss: 1.6423 - accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 1ms/step - loss: 1.5607 - accuracy: 0.3333
Epoch 3/50
1/1 [==============================] - 0s 3ms/step - loss: 1.4949 - accuracy: 0.3333
Epoch 4/50
1/1 [==============================] - 0s 1ms/step - loss: 1.4304 - accuracy: 0.3333
Epoch 5/50
1/1 [==============================] - 0s 3ms/step - loss: 1.3528 - accuracy: 0.3333
Epoch 6/50
1/1 [==============================] - 0s 3ms/step - loss: 1.2520 - accuracy: 0.3333
Epoch 7/50
1/1 [==============================] - 0s 931us/step - loss: 1.1322 - accuracy: 0.5000
Epoch 8/50
1/1 [==============================] - 0s 3ms/step - loss: 1.0034 - accuracy: 0.6667
Epoch 9/50
1/1 [==============================] - 0s 10ms/step - loss: 0.8739 - accuracy: 0.6667
Epoch 10/50
1/1 [==============================] - 0s 11ms/step - loss: 0.7522 - accuracy: 1.0000
Epoch 11/50
1/1 [==============

In [7]:
tf.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn (RNN)                    (None, 6, 5)              220       
_________________________________________________________________
time_distributed (TimeDistri (None, 6, 5)              30        
Total params: 250
Trainable params: 250
Non-trainable params: 0
_________________________________________________________________


In [8]:
predictions = tf.model.predict(x_one_hot)
for i, prediction in enumerate(predictions):
    print(prediction)
    # print char using argmax, dict
    result_str = [idx2char[c] for c in np.argmax(prediction, axis=1)]
    print("\tPrediction str: ", ''.join(result_str))

[[3.3409353e-03 9.9621320e-01 1.7314403e-04 9.3306217e-06 2.6345375e-04]
 [9.9693525e-01 2.6814085e-03 3.7804438e-04 2.3227831e-06 2.9195221e-06]
 [2.6143919e-04 4.3321899e-04 9.9837506e-01 9.2572696e-04 4.5508918e-06]
 [1.1580502e-04 9.3132745e-05 5.7276030e-04 9.9905425e-01 1.6396202e-04]
 [1.1603317e-06 1.8294910e-06 6.5926717e-05 9.9853885e-01 1.3922203e-03]
 [6.3128323e-06 4.4055379e-04 9.8952273e-07 1.0624534e-03 9.9848962e-01]]
	Prediction str:  ihello
